# APVC – Exercícios – Espaços de cor, Binarização

In [2]:
### imports

# ! pip install opencv-python
import cv2
import numpy as np

## Exercício 1 (acesso aos pixels, espaços de cor)

Desenvolva um script em Python que lê uma imagem e calcula o seu “negativo” na parte
da imagem correspondente a um quadrado (ou retângulo) centrado. As dimensões do
quadrado (ou retângulo) correspondem a metade das resoluções da imagem original.

In [3]:
img = cv2.imread("images/lenna.png")
img.shape

(512, 512, 3)

In [4]:
# Definir tamanho do recorte
n = 150

# Clonar a imagem original para modificar
imgMasked = img.copy()

# Coordenadas do centro
x_center, y_center = img.shape[1] // 2, img.shape[0] // 2

# Recortar a região central
imgCrop = img[y_center - n:y_center + n, x_center - n:x_center + n]

# Inverter as cores do recorte
imgCrop = cv2.bitwise_not(imgCrop)

# Substituir a região original pela versão invertida
imgMasked[y_center - n:y_center + n, x_center - n:x_center + n] = imgCrop

# Mostrar as imagens
cv2.imshow("Masked Image", imgMasked)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Exercício 2 (acesso aos pixels, conversões entre espaços de cor)

Pretende-se construir uma imagem que mostra a evolução da tonalidade das cores à
medida que se varia a componente Hue, mantendo a Saturation e Value com valores
constantes (255).

In [5]:
altura = 500
largura = 1080

hue = np.tile(np.linspace(0, 179, largura, dtype=np.uint8), (altura, 1))
hue

array([[  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179],
       ...,
       [  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179]], dtype=uint8)

In [6]:
# Criar matriz para os canais Saturação (S) e Valor (V), fixos em 255
saturation = np.full((altura, largura), 255, dtype=np.uint8)
value = np.full((altura, largura), 255, dtype=np.uint8)

# Juntar os canais H, S e V
hsv_image = cv2.merge([hue, saturation, value])
print(hsv_image)

# Converter para BGR
bgr_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

[[[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 ...

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]]


In [7]:
# Mostrar a imagem
cv2.imshow("Hue Gradient", bgr_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Exercício 3 (espaços de cor, binarização e máscaras)

Utilize máscaras binárias de forma a conseguir que, a partir de uma imagem com múltiplos objetos coloridos, se consiga chegar a uma imagem onde apareçam os objetos de uma determinada cor, ficando o resto da imagem a preto.

### 3.1. De forma automática

In [8]:
# Função para verificar se o valor HUE da matriz está dentro do intervalo de cores
def isColorHue(value, color="blue"):
    if not (0 <= value <= 255):
        raise ValueError("O valor deve estar no intervalo [0, 255].")

    color_ranges = {
        "red": (0, 10) or (170, 255),  # Tons de vermelho estão nos extremos do círculo de matizes
        "orange": (10, 25),
        "yellow": (25, 45),
        "green": (45, 85),
        "cyan": (85, 105),
        "blue": (105, 135),
        "magenta": (135, 165),
        "pink": (165, 185),
        "purple": (185, 200)
    }

    if color not in color_ranges:
        raise ValueError(f"Cor inválida: {color}. Escolha entre {', '.join(color_ranges.keys())}.")

    min_val, max_val = color_ranges[color]

    # Caso especial para o vermelho, que está nos extremos do espectro circular
    if color == "red":
        return (0 <= value <= 10) or (170 <= value <= 255)
    else:
        return min_val <= value <= max_val

def calcular_threshold(array, coluna):
    media = np.mean(array[..., coluna])
    desvio = np.std(array[..., coluna])
    lim_inf = media - desvio  # Limite inferior
    return lim_inf


In [28]:
img = cv2.imread("images/legos.jpg")
img.shape

hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

tresholdSat = calcular_threshold(hsv_img, 1) * 0.65
tresholdVal = calcular_threshold(hsv_img, 2) * 0.65
for i in hsv_img:
    for j in i:
        if not isColorHue(j[0], "red"):
            j[2] = 0
        if j[1] < tresholdSat:
            j[2] = 0
        if j[2] < tresholdVal:
            j[2] = 0
hsv_img

array([[[177, 252, 245],
        [177, 254, 243],
        [177, 255, 244],
        ...,
        [ 26, 243,   0],
        [ 26, 227,   0],
        [ 26, 232,   0]],

       [[177, 252, 243],
        [177, 254, 243],
        [177, 255, 242],
        ...,
        [ 26, 225,   0],
        [ 26, 228,   0],
        [ 26, 233,   0]],

       [[177, 251, 242],
        [177, 254, 239],
        [177, 255, 239],
        ...,
        [ 26, 218,   0],
        [ 26, 229,   0],
        [ 26, 234,   0]],

       ...,

       [[178, 217, 226],
        [179, 197, 234],
        [  2, 219, 200],
        ...,
        [179, 253, 166],
        [  0, 255, 163],
        [  0, 255, 161]],

       [[178, 208, 233],
        [179, 214, 213],
        [  3, 227, 188],
        ...,
        [179, 253, 166],
        [  0, 252, 162],
        [  1, 255, 161]],

       [[179, 218, 223],
        [  0, 237, 189],
        [  3, 220, 188],
        ...,
        [  0, 250, 163],
        [  0, 250, 162],
        [  0, 250, 160]]

In [ ]:
# Converter para BGR
bgr_image = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
cv2.imshow("bgr_image", bgr_image)
cv2.imshow("og_image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 3.1. De forma manual

In [10]:
import cv2
import numpy as np

# Carregar a imagem
img = cv2.imread("images/legos.jpg")

# Converter para HSV
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Dicionário com os intervalos de matizes para cada cor
color_ranges = {
    "red": [(0, 10), (170, 180)],  # Nota: O intervalo superior do vermelho foi ajustado para 180
    "orange": [(10, 25)],
    "yellow": [(25, 45)],
    "green": [(45, 85)],
    "cyan": [(85, 105)],
    "blue": [(105, 135)],
    "magenta": [(135, 165)],
    "pink": [(165, 185)],
    "purple": [(185, 200)]
}

# Função para obter a máscara com base na cor pretendida
def get_color_mask(hsv_img, color):
    if color not in color_ranges:
        raise ValueError("Cor inválida. Escolha uma cor existente.")
    
    masks = []
    for (lower, upper) in color_ranges[color]:
        lower_bound = np.array([lower, 100, 100])
        upper_bound = np.array([upper, 255, 255])
        mask = cv2.inRange(hsv_img, lower_bound, upper_bound)
        masks.append(mask)
    
    full_mask = masks[0]
    if len(masks) > 1:
        for m in masks[1:]:
            full_mask = cv2.bitwise_or(full_mask, m)
    
    return full_mask

# Selecionar a cor pretendida: "blue" (azul)
color = "blue"
mask = get_color_mask(hsv_img, color)

# Aplicar a máscara à imagem original
blue_only = cv2.bitwise_and(img, img, mask=mask)

# Separar os canais HSV (se necessário)
h, s, v = cv2.split(hsv_img)

# Mostrar os resultados
cv2.imshow("Imagem Original", img)
cv2.imshow(f"Máscara {color}", mask)
cv2.imshow(f"Apenas {color}", blue_only)
cv2.waitKey(0)
cv2.destroyAllWindows()

- - -